In [2]:
import numpy as np
import pandas as pd

In [20]:
latent_dim = 100
sigma = 1
sigma_p = 1
sigma_q = 1
mu = 0
rating_path = '../data/ml-latest-small/ratings.csv'


In [17]:
rating_pd = pd.read_csv(rating_path)
num_user = np.unique(rating_pd.values[:, 0]).shape[0]
num_movie = np.unique(rating_pd.values[:, 1]).shape[0]
rating_pd.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [37]:
r_matrix = np.zeros((num_user, num_movie))
p_matrix = np.random.normal(mu, sigma_p, (num_user, latent_dim))
q_matrix = np.random.normal(mu, sigma_q, (num_movie, latent_dim))

movie_dic = {}
idx = 0
for movie_id in np.unique(rating_pd.values[:, 1]):
    movie_dic[movie_id] = idx
    idx += 1

for index, row in rating_pd.iterrows():
    r_matrix[int(row['userId'] - 1), movie_dic[row['movieId']]] = row['rating']

In [45]:
def MSE (r, p, q):
    p_qt_matrix = np.dot(p, q.T)
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return np.sum(result)
    
def L2_loss(sig1, sig2, matrix):
    result = np.power(matrix, 2)
    result = np.sum(result)
    constant = sig1 / float(sig2 * 2)
    return constant * result    

In [46]:
error = MSE(r_matrix, p_matrix, q_matrix) + L2_loss(sigma, sigma_q, q_matrix) + L2_loss(sigma, sigma_p, p_matrix)